In [15]:
%load_ext autoreload
%autoreload 2

import docker
import logging
from jtldockerctl.dctl import *
import hashlib
from names_generator import generate_name
from jtlutil.config import get_config

config = get_config('./config.env')

logging.basicConfig(level=logging.ERROR)
logger.setLevel(logging.ERROR)

client = docker.DockerClient(base_url=config.SSH_URI )

ensure_network_exists(client, 'x11', is_external=False, network_type="bridge")
ensure_network_exists(client, 'caddy', is_external=True, network_type="bridge")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
def make_servers(usernames):
    servers = []
    for username in usernames:
        nvc, pa = create_cs_pair(client, config, config.IMAGES_PYTHONCS, username)
        yield (nvc, pa)
    return servers

print('----')
for nvc, pa in make_servers(['jed','osvaldo', 'eric']):
    print(nvc.id, pa.labels['caddy'])


----
daead746391fd1635ae4b836421b6dce6b1ef580eff6084522db99b92ade545b jed.do.jointheleague.org
deb13fc38d62d4cda951a882eeeee8bf2b579091c4168514a316093d7810a8a0 osvaldo.do.jointheleague.org
d92238a3fd0896fdd116d3872507bad7fdef888eb7ac474dc0abe3e9e2e0b7d8 eric.do.jointheleague.org


In [20]:

containers = client.containers.list(filters={"label": f"jtl"}, all=True)

for c in containers:
    print(c.id, c.labels['caddy'])
    c.remove(force=True)    

418f2d1e1b825ae8138388d4234072c4d82f03184093ac788877df94169a452a eric-busboom-org.do.jointheleague.org
6c17b76099966f8253e99da7b0165a1ecf0bd13b1d17598ad910d948cb209d8e eric-busboom-org-novnc.do.jointheleague.org
d61a1b88e11151793d37f6525a739240f05290979abd5c9321106ed6a76fe23e eric.do.jointheleague.org
d92238a3fd0896fdd116d3872507bad7fdef888eb7ac474dc0abe3e9e2e0b7d8 eric-novnc.do.jointheleague.org
5ba78e2737d264cfaf240bfd2b7864f63b9c0404ed0c54ccd35548a7cb859507 osvaldo.do.jointheleague.org
deb13fc38d62d4cda951a882eeeee8bf2b579091c4168514a316093d7810a8a0 osvaldo-novnc.do.jointheleague.org
c1c107fbf371d62c9a5613f0d1da3dc404ddcc1605e38c02cc8255f8cbadcb05 jed.do.jointheleague.org
daead746391fd1635ae4b836421b6dce6b1ef580eff6084522db99b92ade545b jed-novnc.do.jointheleague.org


In [19]:
pa.labels['caddy']

'eric.do.jointheleague.org'